In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd

from src.custom_dataset import CustomDataset
from src.handler import Handler

In [ ]:
images_data_path = './data/archive/images'
x_train_file_path = './data/x_train.csv'
y_train_file_path = './data/y_train.csv'

In [ ]:
batch_size = 32
num_epochs = 2
checkpoint_interval = 1000

In [ ]:
dataset = CustomDataset(csv_file_path=x_train_file_path, image_folder_path=images_data_path)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
y_train = pd.read_csv(y_train_file_path)

In [ ]:
model = Handler(num_categories_list=dataset.get_num_categories_list())
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    
    for batch_idx, (images, tabular_data) in enumerate(data_loader):
        # Move data to GPU if available
        images, tabular_data = images.to(device), tabular_data.to(device)
        
        # Get the corresponding labels for the current batch
        labels = torch.tensor(y_train.iloc[batch_idx * batch_size: (batch_idx + 1) * batch_size].values).to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images, tabular_data)
        
        # Compute the loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Update running loss and accuracy metrics
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)
        
        # Save model weights periodically
        if total_samples % checkpoint_interval == 0:
            checkpoint_path = f'model_checkpoint_{total_samples}.pth'
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Checkpoint saved at '{checkpoint_path}' for {total_samples} samples.")
        
        # Print training stats for the current batch
        if batch_idx % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx}/{len(data_loader)}], "
                  f"Loss: {loss.item():.4f}, Batch Accuracy: {100 * correct_predictions / total_samples:.2f}%")
    
    # Epoch-level loss and accuracy
    epoch_loss = running_loss / len(data_loader)
    epoch_accuracy = 100 * correct_predictions / total_samples
    print(f"Epoch {epoch+1}/{num_epochs} completed: Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")
    
torch.save(model.state_dict(), 'trained_model.pth')
print("Model saved as 'trained_model.pth'")